Flores Lara Alberto 6BV1

Clasificación para CIFAR usando CNN con Transfer Learning

Librerias necesarias:

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications import VGG16, vgg16
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report
import time
import tensorflow as tf
from PIL import Image


Definir las funciones para crear la CNN con transfer learning y para el entrenamiento y validación del modelo

In [2]:
def preprocess_and_resize(image):
    # Convertir el array a una imagen PIL
    image = Image.fromarray((image * 255).astype(np.uint8))
    # Redimensionar la imagen a 224x224
    image = image.resize((224, 224))
    # Convertir de nuevo a array numpy
    image = np.array(image).astype('float32')
    # Aplicar el preprocesamiento específico de VGG16
    image = vgg16.preprocess_input(image)
    return image

# Función para construir CNN con Transfer Learning
def build_cnn_transfer_learning(input_shape):
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)
    for layer in base_model.layers:
        layer.trainable = False  # Congelar las capas del modelo base

    x = base_model.output
    x = GlobalAveragePooling2D()(x)  # Mejor que Flatten para evitar demasiados parámetros
    x = Dense(256, activation='relu')(x)
    x = tf.keras.layers.Dropout(0.5)(x)  # Añadir Dropout para evitar overfitting
    output_layer = Dense(10, activation='softmax')(x)

    model = Model(inputs=base_model.input, outputs=output_layer)
    return model

def train_and_evaluate(model, train_generator, validation_data, epochs=30, steps_per_epoch=100, validation_steps=50):
    model.compile(optimizer=Adam(learning_rate=0.0001),  # Usar una tasa de aprendizaje más baja
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    start_time = time.time()
    history = model.fit(
        train_generator,
        epochs=epochs,
        steps_per_epoch=steps_per_epoch,
        validation_data=validation_data,
        validation_steps=validation_steps
    )
    end_time = time.time()

    # Evaluación
    y_pred = np.argmax(model.predict(validation_data[0]), axis=1)
    y_true = np.argmax(validation_data[1], axis=1)
    report = classification_report(y_true, y_pred, target_names=[f'Clase {i}' for i in range(10)])
    print(report)
    print(f"Tiempo de entrenamiento: {end_time - start_time:.2f} segundos")

    return history, end_time - start_time

Cargar los datos de entrada

In [ ]:
# Cargar CIFAR-10
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Redimensionar las imágenes a 224x224
x_train_resized = resize_images(x_train, size=(224, 224))
x_test_resized = resize_images(x_test, size=(224, 224))

# Aplicar el preprocesamiento específico de VGG16
x_train_preprocessed = vgg16.preprocess_input(x_train_resized)
x_test_preprocessed = vgg16.preprocess_input(x_test_resized)

# Convertir las etiquetas a one-hot encoding
y_train_cat = to_categorical(y_train, 10)
y_test_cat = to_categorical(y_test, 10)

# Crear generadores de datos con Data Augmentation
train_datagen = ImageDataGenerator(
    rotation_range=15,
    horizontal_flip=True,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1
)

train_generator = train_datagen.flow(x_train_preprocessed, y_train_cat, batch_size=64)

validation_datagen = ImageDataGenerator()  # Sin augmentación para validación
validation_generator = validation_datagen.flow(x_test_preprocessed, y_test_cat, batch_size=64)

Inicializar el modelo con el tamaño de nuestras imagenes y realizar el entrenamiento con la validacion

In [ ]:
# Construir y entrenar el modelo
model_cnn_tl = build_cnn_transfer_learning((224, 224, 3))
history_cnn_tl, time_cnn_tl = train_and_evaluate(
    model_cnn_tl,
    train_generator,
    validation_data=(x_test_preprocessed, y_test_cat),
    epochs=30,
    steps_per_epoch=len(x_train_preprocessed) // 64,
    validation_steps=len(x_test_preprocessed) // 64
)

In [ ]:
# Graficar precisión de entrenamiento y validación
def plot_history(histories, labels):
    plt.figure(figsize=(12, 6))
    for history, label in zip(histories, labels):
        plt.plot(history.history['accuracy'], label=f'{label} - Entrenamiento', linestyle='--')
        plt.plot(history.history['val_accuracy'], label=f'{label} - Validación')
    plt.xlabel('Épocas')
    plt.ylabel('Precisión')
    plt.legend()
    plt.grid(True)
    plt.show()

plot_history([history_cnn_tl], ['CNN Transfer Learning'])
